In [1]:
import numpy as np
import pandas as pd
from typing import List

Fonksiyonların Tanımlanması

AHP İle Kriter Ağırlığı Hesaplanması İçin Kullanılacak Fonksiyon

In [2]:
def ahp_weighting(df: pd.DataFrame, RI: float, excel_path: str) -> np.ndarray:
    # Kriter isimlerini satır ve sütun olarak eşitlenir
    df.columns = df.index

    # Kare matris kontrolü gerçekleştirilir
    if df.shape[0] != df.shape[1]:
        raise ValueError("Karşılaştırma matrisi kare değil. Lütfen satır ve sütun isimlerini kontrol edin.")

    A = df.to_numpy()
    n = A.shape[0]

    # Özdeğer ve özvektör hesaplanır
    eigvals, eigvecs = np.linalg.eig(A)
    max_index = np.argmax(eigvals.real)
    lambda_max = eigvals.real[max_index]
    w = eigvecs[:, max_index].real
    w = w / np.sum(w)

    # Tutarlılık hesapları gerçekleştirilir
    CI = (lambda_max - n) / (n - 1)
    CR = CI / RI
    consistency = "acceptable" if CR < 0.10 else "not acceptable"

    # Hesaplanan ağırlıklar tabloya aktarılır
    weights_df = pd.DataFrame({
        "Kriter": df.index,
        "Ağırlık": np.round(w, 4)
    })

    # CI,RI gibi değişkenler dokümantasyon için tabloya aktarılır
    details_df = pd.DataFrame({
        "Parametre": ["λ_max", "CI", "RI", "CR", "Tutarlılık Durumu"],
        "Değer": [round(lambda_max, 4), round(CI, 4), RI, round(CR, 4), consistency]
    })

    # Oluşturulan Tablolar Dokümantasyon İçin Excel'e Yazdırılır
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Karşılaştırma Matrisi')
        weights_df.to_excel(writer, sheet_name='Ağırlıklar', index=False)
        details_df.to_excel(writer, sheet_name='Parametreler', index=False)

    return w  # Ağırlıklar array olarak döndürülür

In [ ]:
def topsis(df_real_values: pd.DataFrame, weights: np.ndarray, dimension_name: str) -> pd.DataFrame:
    import pandas as pd
    import numpy as np

    #Gerçek vaka  değerlerini al
    real_values = df_real_values["KPI Değeri"].values
    num_criteria = len(real_values)

    #En iyi ve en kötü senaryoları oluştur
    best = np.full(num_criteria, 3)
    worst = np.full(num_criteria, 1)

    #Alternatiflerin birleştirilmesi
    all_data = pd.DataFrame(
        [best, real_values, worst],
        columns=[f"K{i+1}" for i in range(num_criteria)],
        index=["Best", "Real", "Worst"]
    )

    #Vektör normalizasyonu
    norm_data = all_data / np.sqrt((all_data ** 2).sum())

    #Ağırlıklı normalize matris
    weighted_data = norm_data * weights

    #Pozitif ve negatif ideal çözümler
    ideal = weighted_data.max()
    anti_ideal = weighted_data.min()

    #Uzaklıkların hesaplanması
    d_pos = np.sqrt(((weighted_data - ideal) ** 2).sum(axis=1))
    d_neg = np.sqrt(((weighted_data - anti_ideal) ** 2).sum(axis=1))

    #TOPSIS skorları (göreli yakınlık)
    scores = d_neg / (d_pos + d_neg)

    #Sıralama tablosu
    result = pd.DataFrame({
        "Alternatif": ["En İdeal Senaryo", "Gerçek Senaryo", "İdeale En Uzak Senaryo"],
        "TOPSIS_Score": scores
    }).sort_values(by="TOPSIS_Score", ascending=False).reset_index(drop=True)

    # Çıktı dosya adı
    output_excel = f"{dimension_name} - Topsis Calculated Parameters.xlsx"

    # Excel çıktıları
    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        all_data.to_excel(writer, sheet_name='4.2.1 Normalize Öncesi')
        norm_data.to_excel(writer, sheet_name='4.2.1 Normalize Matris')
        weighted_data.to_excel(writer, sheet_name='4.2.2 Ağırlıklı Matris')
        pd.DataFrame([ideal, anti_ideal], index=["Pozitif İdeal", "Negatif İdeal"]).to_excel(writer, sheet_name='4.2.3 İdeal Çözümler')
        pd.DataFrame({"Uzaklık (Pozitif)": d_pos, "Uzaklık (Negatif)": d_neg}, index=["Best", "Real", "Worst"]).to_excel(writer, sheet_name='4.2.4 Uzaklıklar')
        pd.DataFrame({"TOPSIS_Score": scores}, index=["Best", "Real", "Worst"]).to_excel(writer, sheet_name='4.2.5 Yakınlık Skoru')
        result.to_excel(writer, sheet_name='4.2.6 TOPSIS Sıralama', index=False)

    return result

In [ ]:
def gra(df_real_values: pd.DataFrame, weights: np.ndarray, dimension_name: str, rho: float = 0.5) -> pd.DataFrame:
    import pandas as pd
    import numpy as np

    #Gerçek vaka  değerlerini al
    real_values = df_real_values["KPI Değeri"].values
    num_criteria = len(real_values)

    #İdeal ve İdeale en uzak alternatiflerini oluştur
    best = np.full(num_criteria, 3)
    worst = np.full(num_criteria, 1)

    #Alternatifleri birleştir
    all_data = pd.DataFrame(
        [best, real_values, worst],
        columns=[f"K{i+1}" for i in range(num_criteria)],
        index=["En İdeal Senaryo", "Gerçek Senaryo", "İdeale En Uzak Senaryo"]
    )

    #Min Max ile Normalizasyon
    norm_data = (all_data - all_data.min()) / (all_data.max() - all_data.min())

    # 5. Referans (ideal) dizi: En İdeal Senaryo
    reference = norm_data.loc["En İdeal Senaryo"]

    # 6. GRC hesapla (Grey Relational Coefficient)
    delta = np.abs(norm_data - reference)
    delta_min = delta.min().min()
    delta_max = delta.max().max()

    grc = (delta_min + rho * delta_max) / (delta + rho * delta_max)

    # Ağırlıklı GRC matrisinin eldesi
    weighted_grc = grc.mul(weights, axis=1)

    # GRA Skorunun Eldesi
    gra_scores = weighted_grc.sum(axis=1)

    # Sonuçların oluşturulması
    result = pd.DataFrame({
        "Alternatif": grc.index,
        "GRA_Score": gra_scores
    }).sort_values(by="GRA_Score", ascending=False).reset_index(drop=True)

    # 9. Excel çıktısı oluştur
    output_excel = f"{dimension_name} - GRA Calculated Parameters.xlsx"
    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        all_data.to_excel(writer, sheet_name='4.3.1 Normalize Öncesi')
        norm_data.to_excel(writer, sheet_name='4.3.1 Normalize Matris')
        reference.to_frame(name='Referans Değer').to_excel(writer, sheet_name='4.3.2 Referans Seri')
        grc.to_excel(writer, sheet_name='4.3.3 GRC (Ham)')
        weighted_grc.to_excel(writer, sheet_name='4.3.3 Ağırlıklı GRC')
        gra_scores.to_frame(name='GRA_Score').to_excel(writer, sheet_name='4.3.4 GRA Skorları')
        result.to_excel(writer, sheet_name='4.3.5 GRA Sıralama', index=False)

    return result

İkili Karşılaştırma Matrislerinin Dataframe'e Aktarılması

In [5]:
sosyal_matris = pd.read_excel("Sosyal_Pairwise.xlsx",header=None)
cevresel_matris = pd.read_excel("Cevresel_Pairwise.xlsx",header=None)
ekonomik_matris = pd.read_excel("Ekonomik_Pairwise.xlsx",header=None)

In [6]:
cevresel_matris

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.0,0.500000,0.500000,0.500000,0.750000,0.75,0.75,1.00,0.500000,0.333333,...,0.250000,0.291667,0.500000,1.000000,0.250000,0.291667,0.225000,0.333333,0.333333,0.416667
1,2.0,1.000000,0.750000,1.000000,2.500000,2.00,3.00,0.75,1.000000,0.500000,...,0.500000,0.500000,1.000000,1.000000,0.500000,0.500000,0.416667,0.416667,0.416667,1.000000
2,2.0,1.500000,1.000000,1.000000,2.000000,2.00,2.50,1.00,0.750000,0.500000,...,0.500000,0.500000,1.000000,0.750000,0.500000,0.291667,0.500000,0.500000,0.500000,1.000000
3,2.0,1.000000,1.000000,1.000000,2.000000,2.00,3.00,1.00,0.750000,0.750000,...,0.416667,0.416667,1.000000,1.000000,0.500000,0.500000,0.500000,0.416667,0.500000,1.000000
4,1.5,0.416667,0.500000,0.500000,1.000000,1.00,2.50,1.00,0.416667,0.500000,...,0.416667,0.416667,0.500000,1.000000,0.291667,0.208333,0.250000,0.333333,0.266667,0.416667
5,1.5,0.500000,0.500000,0.500000,1.000000,1.00,1.00,0.50,0.416667,0.333333,...,0.250000,0.266667,0.500000,0.416667,0.250000,0.171429,0.250000,0.333333,0.416667,0.416667
6,1.5,0.333333,0.416667,0.333333,0.416667,1.00,1.00,0.50,0.333333,0.225000,...,0.200000,0.208333,0.291667,0.500000,0.200000,0.126984,0.138889,0.250000,0.162500,0.291667
7,1.0,1.500000,1.000000,1.000000,1.000000,2.00,2.00,1.00,0.500000,0.500000,...,0.333333,0.500000,0.500000,1.000000,0.291667,0.333333,0.291667,0.416667,0.333333,0.416667
8,2.0,1.000000,1.500000,1.500000,2.500000,2.50,3.00,2.00,1.000000,1.000000,...,0.500000,1.000000,0.750000,2.000000,0.500000,0.500000,0.500000,1.000000,0.500000,1.000000
9,3.0,2.000000,2.000000,1.500000,2.000000,3.00,4.50,2.00,1.000000,1.000000,...,0.416667,0.750000,1.000000,2.000000,0.416667,0.500000,0.500000,1.000000,1.000000,0.750000


In [7]:
cevresel_agirliklar = ahp_weighting(cevresel_matris,1.6809,"Cevresel_AHP_Parametreleri.xlsx")

In [8]:
cevresel_agirliklar

array([0.01139771, 0.02073733, 0.02090198, 0.0211854 , 0.0130647 ,
       0.01104888, 0.00834656, 0.01574969, 0.02726395, 0.02840315,
       0.0428391 , 0.01317635, 0.01830541, 0.03319123, 0.04656679,
       0.0787031 , 0.08088344, 0.05690739, 0.04841484, 0.01133466,
       0.0469353 , 0.03495388, 0.02718392, 0.01723188, 0.04802122,
       0.05620794, 0.05028793, 0.03740734, 0.04445141, 0.02889752])

In [9]:
sosyal_matris

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,1.000000,4.00,3.5,0.225000,0.133929,0.208334,0.155556,0.250000,0.416667,0.266667,...,0.171429,0.142857,0.142857,0.133929,2.000000,0.171429,0.225000,2.50,2.50,3.50
1,0.250000,1.00,1.5,0.138889,0.111111,0.111111,0.111111,0.111111,0.126984,0.111111,...,0.126984,0.111111,0.111111,0.111111,0.416666,0.111111,0.111111,0.50,1.00,0.75
2,0.291666,0.75,1.0,0.111111,0.111111,0.138889,0.111111,0.126984,0.155556,0.111111,...,0.111111,0.111111,0.111111,0.111111,0.750000,0.126984,0.111111,0.50,1.00,1.00
3,4.500000,7.50,9.0,1.000000,0.500000,0.750000,0.750000,0.666667,2.000000,1.000000,...,1.000000,0.750000,0.750000,0.416666,9.000000,1.000000,1.000000,9.00,8.00,7.00
4,7.500000,9.00,9.0,2.000000,1.000000,1.500000,1.000000,0.750000,1.500000,1.000000,...,0.750000,1.000000,1.000000,0.666667,8.000000,1.500000,1.000000,7.50,9.00,9.00
5,5.000000,9.00,7.5,1.500000,0.750000,1.000000,1.000000,0.750000,1.000000,0.750000,...,1.000000,0.500000,0.750000,0.416666,6.500000,0.750000,1.000000,9.00,7.50,9.00
6,7.000000,9.00,9.0,1.500000,1.000000,1.000000,1.000000,1.000000,1.500000,1.000000,...,1.000000,0.416666,0.500000,0.500000,9.000000,1.000000,0.750000,9.00,7.50,8.00
7,4.500000,9.00,8.0,2.000000,1.500000,1.500000,1.000000,1.000000,1.500000,1.500000,...,1.000000,0.750000,1.000000,0.750000,9.000000,1.000000,0.750000,9.00,9.00,9.00
8,2.500000,8.00,7.0,0.500000,0.750000,1.000000,0.750000,0.750000,1.000000,0.750000,...,0.500000,0.500000,0.500000,0.416667,6.500000,0.500000,0.500000,8.00,8.00,8.50
9,4.000000,9.00,9.0,1.000000,1.000000,1.500000,1.000000,0.750000,1.500000,1.000000,...,0.750000,0.750000,0.750000,0.500000,9.000000,1.000000,1.000000,8.00,9.00,8.00


In [10]:
sosyal_agirliklar = ahp_weighting(sosyal_matris,1.6839,"Sosyal_AHP_Parametreleri.xlsx")

In [11]:
ekonomik_matris

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.000000,0.291667,4.0,0.154762,0.183333,0.126984,0.126984,0.500000,0.142857,3.00,0.111111,0.200000,0.200000,0.118056,0.333333
1,3.500000,1.000000,9.0,0.500000,1.000000,1.000000,0.750000,1.500000,0.500000,8.50,1.000000,0.500000,1.000000,1.000000,1.500000
2,0.333333,0.111111,1.0,0.111111,0.138889,0.111111,0.111111,0.111111,0.111111,0.75,0.111111,0.111111,0.111111,0.111111,0.138889
3,6.500000,2.000000,9.0,1.000000,1.000000,2.000000,2.000000,3.500000,1.000000,9.00,1.000000,0.500000,1.000000,0.500000,2.500000
4,5.500000,1.000000,7.5,1.000000,1.000000,0.500000,0.500000,2.000000,0.291667,9.00,0.500000,0.416667,0.333333,1.000000,2.000000
5,8.000000,1.000000,9.0,0.500000,2.000000,1.000000,1.000000,2.000000,0.500000,8.50,0.500000,1.000000,1.000000,1.000000,2.500000
6,8.000000,1.500000,9.0,0.500000,2.000000,1.000000,1.000000,2.000000,0.333333,7.50,0.416667,1.000000,0.500000,0.750000,3.500000
7,2.000000,0.750000,9.0,0.291667,0.500000,0.500000,0.500000,1.000000,0.500000,7.50,0.500000,0.500000,0.333333,0.500000,1.500000
8,7.000000,2.000000,9.0,1.000000,3.500000,2.000000,3.000000,2.000000,1.000000,9.00,1.000000,1.000000,1.000000,1.000000,5.000000
9,0.375000,0.118056,1.5,0.111111,0.111111,0.118056,0.138889,0.138889,0.111111,1.00,0.111111,0.111111,0.111111,0.111111,0.162500


In [12]:
ekonomik_agirliklar = ahp_weighting(ekonomik_matris,1.5838,"Ekonomik_AHP_Parametreleri.xlsx")

Çok Kriterli Karar Verme İçin Case Verilerinin Dataframelere Aktarılması

In [13]:
Ekonomik_Veriler = pd.read_excel("Ekonomik_Veriler.xlsx")
Sosyal_Veriler = pd.read_excel("Sosyal_Veriler.xlsx")
Cevresel_Veriler = pd.read_excel("Cevresel_Veriler.xlsx")


TOPSIS Yardımı İle Skor Hesaplamaları

In [14]:
Topsis_Ekonomik_Skor = topsis(Ekonomik_Veriler,ekonomik_agirliklar,"Ekonomik")
Topsis_Sosyal_Skor = topsis(Sosyal_Veriler,sosyal_agirliklar,"Sosyal")
Topsis_Cevresel_Skor = topsis(Cevresel_Veriler,cevresel_agirliklar,"Cevresel")

In [15]:
Topsis_Ekonomik_Skor


,Alternatif,TOPSIS_Score
0,En İdeal Senaryo,1.000000
1,Gerçek Senaryo,0.489929
2,İdeale En Uzak Senaryo,0.000000


In [16]:
Topsis_Sosyal_Skor

,Alternatif,TOPSIS_Score
0,En İdeal Senaryo,1.000000
1,Gerçek Senaryo,0.495834
2,İdeale En Uzak Senaryo,0.000000


In [17]:
Topsis_Cevresel_Skor

,Alternatif,TOPSIS_Score
0,En İdeal Senaryo,1.000000
1,Gerçek Senaryo,0.496059
2,İdeale En Uzak Senaryo,0.000000


In [18]:
Topsis_Cevresel_Gercek_Skor =Topsis_Cevresel_Skor.iloc[1][1]
Topsis_Sosyal_Gercek_Skor =Topsis_Sosyal_Skor.iloc[1][1]
Topsis_Ekonomik_Gercek_Skor =Topsis_Ekonomik_Skor.iloc[1][1]
Topsis_Genel_Score = round((Topsis_Sosyal_Gercek_Skor+Topsis_Cevresel_Gercek_Skor+Topsis_Ekonomik_Gercek_Skor)/3,4)

C:\Users\User\AppData\Local\Temp\ipykernel_24928\1526115745.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Topsis_Cevresel_Gercek_Skor =Topsis_Cevresel_Skor.iloc[1][1]
C:\Users\User\AppData\Local\Temp\ipykernel_24928\1526115745.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Topsis_Sosyal_Gercek_Skor =Topsis_Sosyal_Skor.iloc[1][1]
C:\Users\User\AppData\Local\Temp\ipykernel_24928\1526115745.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use 

GRA Yardımı İle Skor Hesaplanması

In [19]:
Gra_Ekonomik_Skor = gra(Ekonomik_Veriler,ekonomik_agirliklar,"Ekonomik")
Gra_Sosyal_Skor = gra(Sosyal_Veriler,sosyal_agirliklar,"Sosyal")
Gra_Cevresel_Skor = gra(Cevresel_Veriler,cevresel_agirliklar,"Cevresel")

In [20]:
Gra_Ekonomik_Skor

,Alternatif,GRA_Score
0,En İdeal Senaryo,1.000000
1,Gerçek Senaryo,0.596365
2,İdeale En Uzak Senaryo,0.333333


In [21]:
Gra_Sosyal_Skor

,Alternatif,GRA_Score
0,En İdeal Senaryo,1.000000
1,Gerçek Senaryo,0.591904
2,İdeale En Uzak Senaryo,0.333333


In [22]:
Gra_Cevresel_Skor

,Alternatif,GRA_Score
0,En İdeal Senaryo,1.000000
1,Gerçek Senaryo,0.694873
2,İdeale En Uzak Senaryo,0.333333


In [23]:
Gra_Cevresel_Gercek_Skor =Gra_Cevresel_Skor.iloc[1][1]
Gra_Sosyal_Gercek_Skor =Gra_Sosyal_Skor.iloc[1][1]
Gra_Ekonomik_Gercek_Skor =Gra_Ekonomik_Skor.iloc[1][1]
Gra_Genel_Score = round((Gra_Sosyal_Gercek_Skor+Gra_Cevresel_Gercek_Skor+Gra_Ekonomik_Gercek_Skor)/3,4)

C:\Users\User\AppData\Local\Temp\ipykernel_24928\3560070491.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Gra_Cevresel_Gercek_Skor =Gra_Cevresel_Skor.iloc[1][1]
C:\Users\User\AppData\Local\Temp\ipykernel_24928\3560070491.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Gra_Sosyal_Gercek_Skor =Gra_Sosyal_Skor.iloc[1][1]
C:\Users\User\AppData\Local\Temp\ipykernel_24928\3560070491.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[po

Son Çıktılar


In [24]:
print(f"Topsis İle Hesaplanan Genel Sürdürülebilirlik Skoru: %{Topsis_Genel_Score*100}")
print(f"Gra İle Hesaplanan Genel Sürdürülebilirlik Skoru: %{Gra_Genel_Score*100}")

Topsis İle Hesaplanan Genel Sürdürülebilirlik Skoru: %49.39
Gra İle Hesaplanan Genel Sürdürülebilirlik Skoru: %62.77
